# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.2.1 with hadoop 3.2, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
    !tar xf spark-3.3.0-bin-hadoop3.tgz
    !mv spark-3.3.0-bin-hadoop3 spark
    !pip install -q findspark

In [ ]:
if IN_COLAB:
  import os
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  os.environ["SPARK_HOME"] = "/content/spark"

# Start a Local Cluster
Use findspark.init() to start a local cluster.  If you plan to use remote cluster, skip the findspark.init() and change the cluster_url according.

In [1]:
import findspark
findspark.init()

In [2]:
spark_url = 'local'

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark ML')\
        .getOrCreate()

22/11/15 01:10:04 WARN Utils: Your hostname, MacBook-Air-khxng-Tanut.local resolves to a loopback address: 127.0.0.1; using 192.168.1.37 instead (on interface en0)
22/11/15 01:10:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/11/15 01:10:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Spark SQL Data Preparation

First, we read a csv file.  We can provide option such as delimiter and header.  We then rename the colume names to remove dot ('.') in the names.

In [5]:
from pyspark.sql.functions import col

In [6]:
path = 'bank-additional-full.csv'

In [7]:
df = spark.read.option("delimiter", ";").option("header", True).csv(path)
cols = [c.replace('.', '_') for c in df.columns]
df = df.toDF(*cols)

In [8]:
cols = ['age', 'duration', 'campaign', 'pdays', 'previous', 'nr_employed']
for c in cols:
    df = df.withColumn(c, col(c).cast('int'))

cols = ['emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m']
for c in cols:
    df = df.withColumn(c, col(c).cast('double'))
    
df = df.withColumn('label', df.y.cast('boolean').cast('int'))

# Split data
We split data into 80% training and 20% testing data

In [9]:
train_df, test_df = df.randomSplit([0.8,0.2])

In [10]:
train_df.count()

32949

In [11]:
test_df.count()

8239

# Spark ML Pipeline
Pipeline is a serie of data transformation to transform data for training and inferring.  A column can contain categorical data or numerical data:
- For categorical data, we have to convert to unique numeric value using **'StringIndexer'** and perform feature encoding with **'OneHotEncoder'**.
- For numerical data, we do not have to do anything.

Once we transform all features, we vectorize them into a single column.

In [12]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

We first setup a pipeline of all data transformation.
- First, we transform all selected string columns
  + use a Transformer, *StringIndexer*, to encode labels in the column to indices (stored in columnnameIndex)
  + perform one hot encoder on the index to map the label index to a binary vector

In [13]:
stages = []

In [14]:
categoricalAttributes = ['job', 'marital', 'education', 'default', 
                         'housing', 'loan', 'contact', 
                         'month', 'day_of_week', 'poutcome']
for columnName in categoricalAttributes:
    stringIndexer = StringIndexer(inputCol=columnName, outputCol=columnName+ "Index")
    stages.append(stringIndexer)
    oneHotEncoder = OneHotEncoder(inputCol=columnName+ "Index", outputCol=columnName + "Vec")
    stages.append(oneHotEncoder)
    
categoricalCols = [s + "Vec" for s in categoricalAttributes]

In [15]:
numericColumns = ['age', 'campaign', 'pdays', 'previous',
           'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 
                  'euribor3m', 'nr_employed']

In [16]:
# Combine all the feature columns into a single column in the dataframe

allFeatureCols =  numericColumns + categoricalCols
vectorAssembler = VectorAssembler(
    inputCols=allFeatureCols,
    outputCol="features")
stages.append(vectorAssembler)

In [17]:
stages

[StringIndexer_6e0a46b30351,
 OneHotEncoder_0c9945f44f85,
 StringIndexer_728a8f495a23,
 OneHotEncoder_8cf8b7724fda,
 StringIndexer_c52e99f71ffa,
 OneHotEncoder_601279b6dc31,
 StringIndexer_10cc82535b39,
 OneHotEncoder_ee9d14861801,
 StringIndexer_86ff9a807efa,
 OneHotEncoder_4428dabd557b,
 StringIndexer_9a1b575b769b,
 OneHotEncoder_ebf619897e36,
 StringIndexer_97aa50e36daa,
 OneHotEncoder_3641dbc1d1b4,
 StringIndexer_b0e6065304ea,
 OneHotEncoder_2970f0ef756f,
 StringIndexer_15b774fdd83b,
 OneHotEncoder_4717c300cd8c,
 StringIndexer_6f8ff9ecd8e0,
 OneHotEncoder_80a3a95c826a,
 VectorAssembler_07953b89fe2f]

# Feature Extraction Pipeline
We build 2 pipelines, feature transformation pipeline and ML pipeline.  This allows us to reuse the feature extraction pipeline with several ML algorithms.  **'fit'** method is called to create a model and we can use **'transform'** to actual transform or infer data

In [18]:
# Build pipeline for feature extraction

featurePipeline = Pipeline(stages=stages)
featureOnlyModel = featurePipeline.fit(train_df)

When we 'fit' a pipeline to the data, we have the model.
As we put only 'Transfomer' in the pipeline, the model is for feature extraction only.

We apply our feature extraction model with 'transform' operation to our training and testing data 
to create new DataFrames with 'features' column that can be used in the next pipeline.

In [19]:
trainingFeaturesDf = featureOnlyModel.transform(train_df)
testFeaturesDf = featureOnlyModel.transform(test_df)

trainingFeaturesDF and testFeaturesDF are training and testing DataFrames with feature columns

In [20]:
trainingFeaturesDf.show(1)

22/11/15 01:13:11 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+-------+-------+---------+-------+-------+----+--------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+-----+--------+---------------+------------+-------------+--------------+-------------+------------+-------------+------------+-------------+---------+-------------+------------+-------------+----------+-------------+----------------+--------------+-------------+-----------+--------------------+
|age|    job|marital|education|default|housing|loan| contact|month|day_of_week|duration|campaign|pdays|previous|poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|label|jobIndex|         jobVec|maritalIndex|   maritalVec|educationIndex| educationVec|defaultIndex|   defaultVec|housingIndex|   housingVec|loanIndex|      loanVec|contactIndex|   contactVec|monthIndex|     monthVec|day_of_weekIndex|day_of_weekVec|poutcomeIndex|poutcomeVec|            features|
+---+-------+-------+-------

In [21]:
set(trainingFeaturesDf.columns) - set(train_df.columns)

{'contactIndex',
 'contactVec',
 'day_of_weekIndex',
 'day_of_weekVec',
 'defaultIndex',
 'defaultVec',
 'educationIndex',
 'educationVec',
 'features',
 'housingIndex',
 'housingVec',
 'jobIndex',
 'jobVec',
 'loanIndex',
 'loanVec',
 'maritalIndex',
 'maritalVec',
 'monthIndex',
 'monthVec',
 'poutcomeIndex',
 'poutcomeVec'}

In [22]:
# Peek into training features

trainingFeaturesDf.select("features", "label").rdd.take(5)

[Row(features=SparseVector(52, {0: 17.0, 1: 3.0, 2: 4.0, 3: 2.0, 4: -2.9, 5: 92.201, 6: -31.4, 7: 0.869, 8: 5076.0, 19: 1.0, 21: 1.0, 25: 1.0, 30: 1.0, 32: 1.0, 34: 1.0, 36: 1.0, 39: 1.0}), label=0),
 Row(features=SparseVector(52, {0: 17.0, 1: 2.0, 2: 999.0, 3: 2.0, 4: -2.9, 5: 92.201, 6: -31.4, 7: 0.869, 8: 5076.0, 19: 1.0, 21: 1.0, 25: 1.0, 30: 1.0, 32: 1.0, 34: 1.0, 36: 1.0, 39: 1.0, 51: 1.0}), label=0),
 Row(features=SparseVector(52, {0: 17.0, 1: 1.0, 2: 2.0, 3: 2.0, 4: -3.4, 5: 92.431, 6: -26.9, 7: 0.742, 8: 5017.0, 19: 1.0, 21: 1.0, 29: 1.0, 30: 1.0, 33: 1.0, 35: 1.0, 36: 1.0, 43: 1.0, 49: 1.0}), label=1),
 Row(features=SparseVector(52, {0: 17.0, 1: 3.0, 2: 4.0, 3: 2.0, 4: -2.9, 5: 92.201, 6: -31.4, 7: 0.884, 8: 5076.0, 19: 1.0, 21: 1.0, 29: 1.0, 30: 1.0, 32: 1.0, 34: 1.0, 36: 1.0, 39: 1.0, 48: 1.0}), label=0),
 Row(features=SparseVector(52, {0: 18.0, 1: 1.0, 2: 999.0, 4: -1.8, 5: 93.075, 6: -47.1, 7: 1.365, 8: 5099.0, 19: 1.0, 21: 1.0, 27: 1.0, 30: 1.0, 33: 1.0, 34: 1.0, 36: 1.0

# Machine Learning Pipeline

Spark ML supports several standard ML algorithm.  In this example, we demonstrate how to use logistic regression and decision tree models.

In [23]:
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier

Calculate accuracy

In [24]:
def calculateAccuracy(results):
    correct = results.filter(results['label'] == results['prediction']).count()
    total = results.count()
    return 1.0*correct/total

## Logistic Regression Model
Configure an machine learning pipeline, which consists of only one stage containing an estimator (classification) (Logistic regression in this case)

In [25]:
lr = LogisticRegression(maxIter=10, regParam=0.01)
lrPipeline = Pipeline(stages=[lr])

Fit the pipeline to create a model from the training data.  The logistic regression estimator looks for column 'features' and 'labels' to create an ML model.

In [26]:
lrPipelineModel = lrPipeline.fit(trainingFeaturesDf)

22/11/15 01:14:42 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/11/15 01:14:42 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


With our trained model, we transform testFeaturesDf to predict the results.  The predicted results are stored in 'prediciton' column.  We then use our calculateAccuracy function to calculate the results.

In [27]:
results = lrPipelineModel.transform(testFeaturesDf)
print('LogisticRegression Model test accuracy = ', calculateAccuracy(results))

LogisticRegression Model test accuracy =  0.9026580895739774


In [28]:
results.select('label', 'prediction').rdd.take(5)

[Row(label=1, prediction=0.0),
 Row(label=0, prediction=0.0),
 Row(label=1, prediction=0.0),
 Row(label=0, prediction=0.0),
 Row(label=0, prediction=0.0)]

## DecisionTree Model

Once again a ML pipeline is created with only an estimator in the pipeline.  We then fit the pipeline with the trainingFeaturesDf to train a model.  Then, we transform testFeaturesDf to predict the results.

In [29]:
dt = DecisionTreeClassifier(labelCol='label', featuresCol='features')
dtPipeline = Pipeline(stages=[dt])

In [30]:
dtPipelineModel = dtPipeline.fit(trainingFeaturesDf)

In [31]:
results = dtPipelineModel.transform(testFeaturesDf)
print('DecisionTree Model test accuracy = ', calculateAccuracy(results))

DecisionTree Model test accuracy =  0.9079985435125622
